In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
#import geopandas as gpd
from shapely.geometry import Polygon
import h3
#import s2
from geojson import Feature, Point, FeatureCollection
import numpy as np
#from no_smoothing_trees.ipynb import df
import pandas as pd

ModuleNotFoundError: No module named 'shapely'

In [2]:
df_vis = pd.read_csv('../../data/samples/tree_data_sample.csv')
df_vis.dropna(subset=['lat', 'lon'], inplace=True)
df_vis.head()

,date,lat,lon,municipality,taxonId,scientificName,verbatim
0,2023-09-13,60.29276,24.56223,Espoo,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris
1,2023-09-11,62.90000,23.30000,"Kuortane, Lapua, Seinäjoki",http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris
2,2023-09-10,61.34076,22.13714,Harjavalta,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris
3,2023-09-09,60.18495,24.93093,Helsinki,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris
4,2023-09-09,60.18457,24.93013,Helsinki,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris


In [3]:
df_vis['municipality'].value_counts()

municipality
Inari                                 4667
Kuusamo                               3349
Suomussalmi                           2633
Sodankylä                             2341
Kuhmo                                 2217
                                      ... 
Loviisa, Porvoo                          1
Iitti, Kouvola                           1
Kaarina, Lieto, Marttila, Paimio         1
Keitele, Pielavesi, Tervo, Vesanto       1
Kemi, Keminmaa, Simo                     1
Name: count, Length: 431, dtype: int64

In [4]:
def geo_to_h3(row):
  return h3.geo_to_h3(lat=row['lat'], lng=row['lon'], resolution = 5)

df_vis['h3_cell'] = df_vis.apply(geo_to_h3, axis=1)
df_vis_ag = df_vis.reset_index(drop=False)
df_vis_ag

,index,date,lat,lon,municipality,taxonId,scientificName,verbatim,h3_cell
0,0,2023-09-13,60.292760,24.562230,Espoo,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85089963fffffff
1,1,2023-09-11,62.900000,23.300000,"Kuortane, Lapua, Seinäjoki",http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85088967fffffff
2,2,2023-09-10,61.340760,22.137140,Harjavalta,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85088a97fffffff
3,3,2023-09-09,60.184950,24.930930,Helsinki,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,851126d3fffffff
4,4,2023-09-09,60.184570,24.930130,Helsinki,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,851126d3fffffff
...,...,...,...,...,...,...,...,...,...
113077,115717,NaN,60.879859,22.271316,Pöytyä,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85088ad3fffffff
113078,115718,NaN,60.920732,22.412991,Pöytyä,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85088ac3fffffff
113079,115719,NaN,66.403813,29.151553,Kuusamo,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,8508d8b7fffffff
113080,115720,NaN,60.858113,22.421991,Pöytyä,http://tun.fi/MX.37819,Pinus sylvestris,Pinus sylvestris,85088ac3fffffff


In [5]:
df_vis_agg = (df_vis_ag
              .groupby('h3_cell')
              .index
              .agg(list)
              .to_frame("ids")
              .reset_index())

df_vis_agg['count'] = df_vis_agg['ids'].apply(lambda row: len(row))
df_vis_agg

,h3_cell,ids,count
0,85012613fffffff,"[9562, 78059, 109502, 113234]",4
1,8501261bfffffff,"[473, 9335, 9345, 9426, 9427, 9485, 9510, 9536...",27
2,85012643fffffff,"[76368, 107634]",2
3,85012647fffffff,"[76362, 76369, 107631, 109126]",4
4,8501264bfffffff,"[102955, 107707, 109417]",3
...,...,...,...
2095,852a33dbfffffff,"[58012, 58013, 67131, 68160, 68165]",5
2096,852aa38ffffffff,"[71939, 71940]",2
2097,852ec31bfffffff,"[67902, 69939, 69974, 70518, 70685, 71261, 714...",56
2098,85314293fffffff,"[76219, 76220]",2


In [6]:
def add_geometry(row):
  points = h3.h3_to_geo_boundary(row['h3_cell'], True)
  return Polygon(points)

df_vis_agg['geometry'] = df_vis_agg.apply(add_geometry,axis=1)
df_vis_agg

,h3_cell,ids,count,geometry
0,85012613fffffff,"[9562, 78059, 109502, 113234]",4,"POLYGON ((26.61443341995143 69.81869000942822,..."
1,8501261bfffffff,"[473, 9335, 9345, 9426, 9427, 9485, 9510, 9536...",27,POLYGON ((27.004894867595613 69.84504909290897...
2,85012643fffffff,"[76368, 107634]",2,"POLYGON ((27.790703442725356 69.8952999863398,..."
3,85012647fffffff,"[76362, 76369, 107631, 109126]",4,"POLYGON ((27.48442136864743 69.99426446576476,..."
4,8501264bfffffff,"[102955, 107707, 109417]",3,"POLYGON ((28.18599647709223 69.91917599268866,..."
...,...,...,...,...
2095,852a33dbfffffff,"[58012, 58013, 67131, 68160, 68165]",5,POLYGON ((-71.77529521858946 42.41666446319952...
2096,852aa38ffffffff,"[71939, 71940]",2,POLYGON ((-77.8941664681639 40.898349144280054...
2097,852ec31bfffffff,"[67902, 69939, 69974, 70518, 70685, 71261, 714...",56,POLYGON ((136.22192851600084 45.38346735470498...
2098,85314293fffffff,"[76219, 76220]",2,"POLYGON ((128.1179458175917 42.02296383870008,..."


In [7]:
def hex_to_geojson(df_hex, hex_id_field, geometry_field, value_field):
  list_features = []
  for i, row in df_hex.iterrows():
    feature = Feature(geometry=row[geometry_field],
                      id=row[hex_id_field],
                      properties={'value': row[value_field]})
    list_features.append(feature)
    feat_collection = FeatureCollection(list_features)
  return feat_collection

geojson_obj = hex_to_geojson(df_vis_agg, hex_id_field='h3_cell', value_field='count', geometry_field='geometry')

In [8]:
def plot_observation_count(data: pd.DataFrame, category: str = 'all'):
  if category != 'all':
    data = data[data['scientificName'] == category]

  month_count = data['date'].apply(lambda row: row[:7] if isinstance(row, str) else None).value_counts().sort_index()
  fig = px.line(x=month_count.index, y=month_count.values)
  fig.update_layout(title=f'Observations count for {category}',
                    xaxis_title='Month',
                    yaxis_title='Observations')
  fig.show()

plot_observation_count(df_vis)

In [9]:
def plot_observation_municipality(data: pd.DataFrame, category: str = 'all'):
  if category != 'all':
    data = data[data['scientificName'] == category]

  mun_count = data['municipality'].value_counts().head(10)
  fig = px.bar(x=mun_count.index, y=mun_count.values)
  fig.update_layout(title=f'Top 10 municipality for {category}',
                    xaxis_title='Municipality',
                    yaxis_title='Observations')
  fig.show()

plot_observation_municipality(df_vis)

In [10]:
def plot_observation_comp(data: pd.DataFrame):
  obs_count = data['scientificName'].value_counts().to_frame().reset_index()
  obs_count.columns = ['scientificName', 'count']
  obs_count_small = obs_count[obs_count['count'] < 250].sum().to_frame().T
  obs_count_small.loc[0, 'scientificName'] = 'Others'
  obs_count_agg = pd.concat([obs_count_small, obs_count[obs_count['count'] >= 250]])
  fig = px.pie(obs_count_agg, values='count', names='scientificName')
  fig.update_layout(title=f'Distributions of composing species')
  fig.show()

plot_observation_comp(df_vis)

In [12]:
def discrete_colorscale(bvals, colors):
    """
    bvals - list of values bounding intervals/ranges of interest
    colors - list of rgb or hex color codes for values in [bvals[k], bvals[k+1]],0<=k < len(bvals)-1
    returns a nonuniform discrete colorscale
    """
    if len(bvals) != len(colors)+1:
        raise ValueError('len(boundary values) should be equal to  len(colors)+1')
    bvals = sorted(bvals)
    nvals = [(v-bvals[0])/(bvals[-1]-bvals[0]) for v in bvals]  #normalized values

    dcolorscale = [] #discrete colorscale
    for k in range(len(colors)):
        dcolorscale.extend([[nvals[k], colors[k]], [nvals[k+1], colors[k]]])
    return dcolorscale

bvals= [1, 10, 100, 1000]
colors=["#e62bf0", "#2026e3", "#26de26"]
discrete_nonuniform = discrete_colorscale(bvals, colors)

bvals = np.array(bvals)
tickvals = [np.mean(bvals[k:k+2]) for k in range(len(bvals)-1)]
ticktext =  [f'{bvals[k]}-' for k in range(0, len(bvals)-1)]

fig = px.choropleth_mapbox(
    df_vis_agg,
    geojson=geojson_obj,
    color_continuous_scale=discrete_nonuniform,
    locations='h3_cell',
    color='count',
    center=dict(lat=65, lon=24),
    zoom=4.3,
    width=600,
    height=650,
    opacity=0.3,
    labels={'count': 'observations'},
    mapbox_style="open-street-map")

fig.update_geos(projection_type='foucaut')

fig.update_layout(
      autosize=False,
      margin = dict(l=0, r=0, b=0, t=0, pad=4, autoexpand=True),
      coloraxis =dict(colorbar_thickness=25, colorbar_ticktext=ticktext, colorbar_tickvals=tickvals)
    )
fig.show()